**Prepare data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Load data**

In [ ]:
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
import os
import json
from torch.utils.data import Dataset

class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, colormap_file, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        with open(colormap_file) as f:
            self.colormap = json.load(f)
            self.color2label = {tuple(color): idx for idx, color in enumerate(self.colormap.values())}
        self.images = os.listdir(image_dir)
        print(f"Found {len(self.images)} images.")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name.replace('.jpg', '.png'))  # Adjust if necessary
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("RGB")
        mask = self.rgb_to_mask(mask)

        if self.transform is not None:
            image = self.transform(image)
            mask = torch.from_numpy(mask).long()

        # Debug statements
        print(f"Loading image: {img_name}")
        print(f"Image shape: {image.shape}")
        print(f"Mask unique values: {torch.unique(mask)}")

        return image, mask

    def rgb_to_mask(self, mask):
        """Convert a RGB mask to a class map mask."""
        mask = np.array(mask)
        class_map = np.zeros(mask.shape[:2], dtype=np.int32)

        for rgb, class_id in self.color2label.items():
            equality = np.equal(mask, rgb)
            class_map[np.all(equality, axis=-1)] = class_id

        # Debug statement
        print(f"Unique classes in mask: {np.unique(class_map)}")

        return class_map

transform = transforms.Compose([
    transforms.ToTensor(),
])

image_dir = '/content/drive/MyDrive/AIP Assignment 2/Question 2/dataset/images'
mask_dir = '/content/drive/MyDrive/AIP Assignment 2/Question 2/dataset/masks'
colormap_file = '/content/drive/MyDrive/AIP Assignment 2/Question 2/dataset/label2cmap.json'

# Create the dataset
dataset = SegmentationDataset(image_dir, mask_dir, colormap_file, transform=transform)

# Optionally, load and inspect a single sample for debugging
image, mask = dataset[0]  # Adjust the index if you want to inspect a different sample

Found 298 images.
Unique classes in mask: [0 1 5 6 7]
Loading image: 2022-08-24 (185).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 5, 6, 7])


**Split training and testing data**

In [ ]:
# Adjust the path to where you have stored 'train_test_split.json' in your Google Drive
split_file_path = '/content/drive/MyDrive/AIP Assignment 2/Question 2/dataset/train_test_split.json'

# Read the JSON file
with open(split_file_path, 'r') as file:
    split_data = json.load(file)

train_images = split_data['train']
test_images = split_data['test']

print(f"Total training images: {len(train_images)}")
print(f"Total testing images: {len(test_images)}")


Total training images: 248
Total testing images: 50


In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, colormap_file, subset, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        with open(colormap_file) as f:
            self.colormap = json.load(f)
            self.color2label = {tuple(color): idx for idx, color in enumerate(self.colormap.values())}
        # Use only the subset of images specified ('train' or 'test')
        if subset == 'train':
            self.images = [img for img in os.listdir(image_dir) if img in train_images]
        elif subset == 'test':
            self.images = [img for img in os.listdir(image_dir) if img in test_images]
        else:
            raise ValueError("Subset must be either 'train' or 'test'")
        print(f"Found {len(self.images)} {subset} images.")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name.replace('.jpg', '.png'))  # Adjust if necessary
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("RGB")
        mask = self.rgb_to_mask(mask)

        if self.transform is not None:
            image = self.transform(image)
            mask = torch.from_numpy(mask).long()

        # Debug statements
        print(f"Loading image: {img_name}")
        print(f"Image shape: {image.shape}")
        print(f"Mask unique values: {torch.unique(mask)}")

        return image, mask

    def rgb_to_mask(self, mask):
        """Convert a RGB mask to a class map mask."""
        mask = np.array(mask)
        class_map = np.zeros(mask.shape[:2], dtype=np.int32)

        for rgb, class_id in self.color2label.items():
            equality = np.equal(mask, rgb)
            class_map[np.all(equality, axis=-1)] = class_id

        # Debug statement
        print(f"Unique classes in mask: {np.unique(class_map)}")

        return class_map

In [ ]:
# Create the training dataset
train_dataset = SegmentationDataset(image_dir, mask_dir, colormap_file, 'train', transform=transform)

# Create the testing dataset
test_dataset = SegmentationDataset(image_dir, mask_dir, colormap_file, 'test', transform=transform)

Found 248 train images.
Found 50 test images.


**Data loaders**

In [ ]:
from torch.utils.data import DataLoader

# Set the batch size for the data loaders
batch_size = 5

# Create the training data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Create the testing data loader
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Verify the data loaders
print(f"Total train batches: {len(train_loader)}")
print(f"Total test batches: {len(test_loader)}")

Total train batches: 50
Total test batches: 10


**Model - with skip connections**

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class FCNResNet18(nn.Module):
    def __init__(self, num_classes):
        super(FCNResNet18, self).__init__()
        # Load a pre-trained ResNet-18 model
        resnet18 = models.resnet18(pretrained=True)

        # Use the features of ResNet-18 up to the final convolutional layer, excluding the average pooling and fully connected layers
        self.features = nn.Sequential(*list(resnet18.children())[:-2])

        # Let's use the output after the second block (layer1 in ResNet terms), which has 64 output channels.
        self.skip_conv = nn.Conv2d(64, num_classes, kernel_size=1)

        # Final classifier that converts feature maps to predictions
        self.classifier = nn.Conv2d(512, num_classes, kernel_size=1)

        # Upsampling layer to match the input image size
        self.upsample = nn.Upsample(scale_factor=32, mode='bilinear', align_corners=False)

    def forward(self, x):
        # Save the input size to upsample the final output back to this size
        input_size = x.size()[2:]

        # Initial conv and bn and relu and maxpool, then through the first two blocks (layers)
        x_initial = self.features[0](x)
        x = self.features[1](x_initial)  # Layer 1 where we'll attach the skip connection

        # Prepare the skip connection, adjusting its channels to match the number of classes
        skip_connection = self.skip_conv(x_initial)

        # Continue through the rest of the ResNet features
        for i in range(2, len(self.features)):
            x = self.features[i](x)

        # Classifier to get the class scores
        x = self.classifier(x)

        # Upsample the output to match the input size
        x = nn.functional.interpolate(x, size=input_size, mode='bilinear', align_corners=False)

        # Add the skip connection
        x += nn.functional.interpolate(skip_connection, size=input_size, mode='bilinear', align_corners=False)

        return x

**Training the model**

In [ ]:
import torch
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm

model = FCNResNet18(num_classes=9)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Specify the loss function.
criterion = torch.nn.CrossEntropyLoss()

# Number of epochs
num_epochs = 1

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for images, true_masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        # Move the inputs and targets to the correct device
        images = images.to(device)
        true_masks = true_masks.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate the loss
        loss = criterion(outputs, true_masks)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print average loss for the epoch
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch: {epoch+1}, Loss: {epoch_loss:.4f}")

print("Training complete")

Epoch 1/1:   0%|          | 0/50 [00:00<?, ?it/s]

Unique classes in mask: [0 1 6 7]
Loading image: 2022-08-24 (209).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 6, 7])
Unique classes in mask: [1 6]
Loading image: 2022-08-24 (165).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([1, 6])
Unique classes in mask: [0 1 6 7]
Loading image: 2022-08-24 (175).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 6, 7])
Unique classes in mask: [0 1 2 3 4 5 6 7]
Loading image: 2022-08-24 (191).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 2, 3, 4, 5, 6, 7])
Unique classes in mask: [1 2 3 4 5 7 8]
Loading image: 2022-08-24 (305).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([1, 2, 3, 4, 5, 7, 8])
Unique classes in mask: [0 1 6 7]
Loading image: 2022-08-24 (59).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 6, 7])
Unique classes in mask: [1 2 3 5 6 8]
Loading image: 2022-08-24 

Epoch 1/1:   2%|▏         | 1/50 [00:11<09:02, 11.06s/it]

Unique classes in mask: [0 1 2 3 4 5 7 8]
Loading image: 2022-08-24 (168).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 2, 3, 4, 5, 7, 8])


Epoch 1/1:   4%|▍         | 2/50 [00:11<03:56,  4.94s/it]

...

Unique classes in mask: [1 2 3 4 5 7 8]
Loading image: 2022-08-24 (297).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([1, 2, 3, 4, 5, 7, 8])
Unique classes in mask: [0 1 6 7]
Loading image: 2022-08-24 (26).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 6, 7])
Unique classes in mask: [1 2 3 4 5 6 8]
Loading image: 2022-08-24 (232).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([1, 2, 3, 4, 5, 6, 8])
Unique classes in mask: [0 1 2 3 5 6 7 8]
Loading image: 2022-08-24 (87).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 2, 3, 5, 6, 7, 8])
Unique classes in mask: [1 2 3 4 5 7 8]
Loading image: 2022-08-24 (171).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([1, 2, 3, 4, 5, 7, 8])
Unique classes in mask: [0 1 6 7]
Unique classes in mask: [0 1 2 3 8]
Loading image: 2022-08-24 (11).png
Loading image: 2022-08-24 (128).pngImage shape: torch.Size([3, 1080, 192

Epoch 1/1:  98%|█████████▊| 49/50 [03:45<00:03,  3.73s/it]

Unique classes in mask: [0 1 2 3 4 5 6 7 8]
Loading image: 2022-08-24 (194).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])
Unique classes in mask: [0 1 4 6 7 8]
Loading image: 2022-08-24 (14).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 4, 6, 7, 8])
Unique classes in mask: [1 2 3 4 7 8]
Loading image: 2022-08-24 (351) - Copy.png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([1, 2, 3, 4, 7, 8])


Epoch 1/1: 100%|██████████| 50/50 [03:50<00:00,  4.60s/it]

Epoch: 1, Loss: 0.7334
Training complete


**Testing**

In [ ]:
import numpy as np
import torch

def pixel_accuracy(output, mask):
    with torch.no_grad():
        predictions = torch.argmax(output, dim=1)
        correct = (predictions == mask).float()
        accuracy = correct.sum() / correct.numel()
    return accuracy.item()

def iou(output, mask, num_classes):
    with torch.no_grad():
        predictions = torch.argmax(output, dim=1)
        iou_list = []
        for cls in range(num_classes):
            true_positive = ((predictions == cls) & (mask == cls)).sum().item()
            false_positive = ((predictions == cls) & (mask != cls)).sum().item()
            false_negative = ((predictions != cls) & (mask == cls)).sum().item()

            if true_positive + false_positive + false_negative > 0:
                iou = true_positive / (true_positive + false_positive + false_negative)
            else:
                iou = float('nan')  # Undefined IoU for this class due to division by zero
            iou_list.append(iou)

        # Filter out 'nan' values to avoid affecting the mean IoU
        iou_list = [x for x in iou_list if not np.isnan(x)]
        mean_iou = np.nanmean(iou_list) if len(iou_list) > 0 else float('nan')
    return mean_iou

In [ ]:
model.eval()  # Set the model to evaluation mode
total_accuracy = 0.0
total_iou = 0.0
num_batches = 0

for images, masks in test_loader:
    images, masks = images.to(device), masks.to(device)

    with torch.no_grad():
        output = model(images)

    acc = pixel_accuracy(output, masks)
    miou = iou(output, masks, 9)

    if not np.isnan(miou):
        total_accuracy += acc
        total_iou += miou
        num_batches += 1

# Calculate mean values
mean_accuracy = total_accuracy / num_batches
mean_iou = total_iou / num_batches

print(f"Mean Pixel Accuracy: {mean_accuracy}")
print(f"Mean IoU: {mean_iou}")

Unique classes in mask: [0 1 6 7]
Unique classes in mask: [1 2 3 4 5 7 8]
Loading image: 2022-08-24 (160).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 6, 7])
Loading image: 2022-08-24 (205).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([1, 2, 3, 4, 5, 7, 8])
Unique classes in mask: [0 1 3 4 6 7 8]
Loading image: 2022-08-24 (280).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 3, 4, 6, 7, 8])
Unique classes in mask: [0 1 6]
Loading image: 2022-08-24 (29).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 6])
Unique classes in mask: [1 2 3 4 5 8]
Loading image: 2022-08-24 (111).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([1, 2, 3, 4, 5, 8])
Unique classes in mask: [0 1 6]
Loading image: 2022-08-24 (135).png
Image shape: torch.Size([3, 1080, 1920])
Mask unique values: tensor([0, 1, 6])
Unique classes in mask: [1 2 3 4 5 8]
Loading image: 2022-0